# Character-Level Neural Machine Translation Using seq2seq & fra-eng dataset

In [43]:
from IPython.display import HTML, display
def set_css():
  display(HTML('''
  <style>
    pre {white-space: pre-wrap;}
  </style>
  '''))
get_ipython().events.register("pre_run_cell", set_css)
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
import os
import sys
try:
    my_path = "/content/notebooks"
    os.symlink("/content/drive/MyDrive/ColabNotebooks/my_env", my_path)
    sys.path.insert(0, my_path)
except:
    pass
os.chdir(my_path)
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, Bidirectional, TimeDistributed, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import Input, Model, Sequential
import matplotlib.pyplot as plt
import seaborn as sb
import pandas as pd
import numpy as np
import re
import urllib3
import zipfile
import shutil
import json
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu, SmoothingFunction

Mounted at /content/drive


- 시퀀스-투-시퀀스(seq-to-seq)는 입력된 시퀀스로부터 다른 도메인의 시퀀스를 출력하는 다양한 분야에서 사용되는 모델입니다. 예를 들어 챗봇(Chatbot)과 기계 번역(Machine Translation)이 그러한 대표적인 예인데, 입력 시퀀스와 출력 시퀀스를 각각 질문과 대답으로 구성하면 챗봇으로 만들 수 있고, 입력 시퀀스와 출력 시퀀스를 각각 입력 문장과 번역 문장으로 만들면 번역기로 만들 수 있습니다. 그 외에도 내용 요약(Text Summarization), STT(Speech to Text) 등에서 쓰일 수 있습니다.

In [4]:
# url ="http://www.manythings.org/anki/fra-eng.zip"
file = "fra-eng.zip"
file_dir = os.path.join(my_path, file)
# shutil.copyfileobj(urllib3.PoolManager().request("GET", url, preload_content=False), open(file_dir, "wb"))

with zipfile.ZipFile(file_dir, "r") as f:
    file = f.open("fra.txt")
    data = pd.read_csv(file, names=["src", "tar", "CC"], sep="\t")
data = data[["src", "tar"]]
data = data.sample(60000, random_state=777) # 6만개만 저장
data = data.reset_index(drop=True)
data["src"] = data["src"].str.lower()
data["tar"] = data["tar"].str.lower()

- ![seq2seq](https://wikidocs.net/images/page/24996/%EC%9D%B8%EC%BD%94%EB%8D%94%EB%94%94%EC%BD%94%EB%8D%94%EB%AA%A8%EB%8D%B8.PNG)
- seq2seq는 크게 두 개로 구성된 아키텍처로 구성되는데, 바로 인코더와 디코더입니다. 인코더는 입력 문장의 모든 단어들을 순차적으로 입력받은 뒤에 마지막에 이 모든 단어 정보들을 압축해서 하나의 벡터로 만드는데, 이를 컨텍스트 벡터(context vector)라고 합니다. 입력 문장의 정보가 하나의 컨텍스트 벡터로 모두 압축되면 인코더는 컨텍스트 벡터를 디코더로 전송합니다. 디코더는 컨텍스트 벡터를 받아서 번역된 단어를 한 개씩 순차적으로 출력합니다.
- 디코더는 초기 입력으로 문장의 시작을 의미하는 심볼 `<sos>`가 들어갑니다. 디코더는 `<sos>`가 입력되면, 다음에 등장할 확률이 높은 단어를 예측합니다. 첫번째 시점(time step)의 디코더 RNN 셀은 다음에 등장할 단어로 je를 예측하였습니다. 첫번째 시점의 디코더 RNN 셀은 예측된 단어 je를 다음 시점의 RNN 셀의 입력으로 입력합니다. 그리고 두번째 시점의 디코더 RNN 셀은 입력된 단어 je로부터 다시 다음에 올 단어인 suis를 예측하고, 또 다시 이것을 다음 시점의 RNN 셀의 입력으로 보냅니다. 디코더는 이런 식으로 기본적으로 다음에 올 단어를 예측하고, 그 예측한 단어를 다음 시점의 RNN 셀의 입력으로 넣는 행위를 반복합니다. 이 행위는 문장의 끝을 의미하는 심볼인 `<eos>`가 다음 단어로 예측될 때까지 반복됩니다. 지금 설명하는 것은 테스트 과정 동안의 이야기입니다.

In [5]:
chars_src = set([char for sent in data["src"] for char in sent])
chars_tar = set([char for sent in data["tar"] for char in sent])

char2idx_src = {}
char2idx_src.update({char:idx+1 for idx, char in enumerate(chars_src)})
idx2char_src = {value:key for key, value in char2idx_src.items()}

char2idx_tar = {}
char2idx_tar["<SOS>"] = 1
char2idx_tar["<EOS>"] = 2
char2idx_tar.update({char:idx+3 for idx, char in enumerate(chars_tar)})
idx2char_tar = {value:key for key, value in char2idx_tar.items()}

- 정상적으로 정수 인코딩이 수행된 것을 볼 수 있습니다. 아직 정수 인코딩을 수행해야 할 데이터가 하나 더 남았습니다. 디코더의 예측값과 비교하기 위한 실제값이 필요합니다. 그런데 이 실제값에는 시작 심볼에 해당되는 `<sos>`가 있을 필요가 없습니다. 이해가 되지 않는다면 이전 페이지의 그림으로 돌아가 Dense와 Softmax 위에 있는 단어들을 다시 보시기 바랍니다. 그래서 이번에는 정수 인코딩 과정에서 `<sos>`를 제거합니다. 즉, 모든 프랑스어 문장의 맨 앞에 붙어있는 '\t'를 제거하도록 합니다

In [6]:
enc_input = data["src"].apply(lambda x:[char2idx_src[char] for char in x]).tolist()
dec_input = data["tar"].apply(lambda x:[1]+[char2idx_tar[char] for char in x]).tolist()
dec_true = data["tar"].apply(lambda x:[char2idx_tar[char] for char in x]+[2]).tolist()

In [11]:
lens_enc = sorted([len(doc) for doc in enc_input])
ratio = 0.99
max_len_enc = int(np.quantile(lens_enc, ratio))
print(f"가장 긴 문장의 길이는 {np.max(lens_enc)}입니다.")
print(f"길이가 {max_len_enc} 이하인 문장이 전체의 {ratio:.0%}를 차지합니다.")

가장 긴 문장의 길이는 239입니다.
길이가 71 이하인 문장이 전체의 99%를 차지합니다.


In [12]:
lens_dec = sorted([len(doc) for doc in dec_input])
ratio = 0.99
max_len_dec = int(np.quantile(lens_dec, ratio))
print(f"가장 긴 문장의 길이는 {np.max(lens_dec)}입니다.")
print(f"길이가 {max_len_dec} 이하인 문장이 전체의 {ratio:.0%}를 차지합니다.")

가장 긴 문장의 길이는 306입니다.
길이가 87 이하인 문장이 전체의 99%를 차지합니다.


In [13]:
enc_input = tf.keras.preprocessing.sequence.pad_sequences(enc_input, padding="post",
                                                          maxlen=max_len_enc)
dec_input = tf.keras.preprocessing.sequence.pad_sequences(dec_input, padding="post",
                                                          maxlen=max_len_dec)
dec_true = tf.keras.preprocessing.sequence.pad_sequences(dec_true, padding="post",
                                                         maxlen=max_len_dec)

enc_input = tf.keras.utils.to_categorical(enc_input)
dec_input = tf.keras.utils.to_categorical(dec_input)
dec_true = tf.keras.utils.to_categorical(dec_true)

- ![rnn](https://wikidocs.net/images/page/24996/rnn%EA%B7%BC%ED%99%A9.PNG)
- 이미 RNN에 대해서 배운 적이 있지만, 다시 복습을 해보도록 하겠습니다. 하나의 RNN 셀은 각각의 시점(time step)마다 두 개의 입력을 받습니다.
- 현재 시점(time step)을 t라고 할 때, RNN 셀은 t-1에서의 은닉 상태와 t에서의 입력 벡터를 입력으로 받고, t에서의 은닉 상태를 만듭니다. 이때 t에서의 은닉 상태는 바로 위에 또 다른 은닉층이나 출력층이 존재할 경우에는 위의 층으로 보내거나, 필요없으면 값을 무시할 수 있습니다. 그리고 RNN 셀은 다음 시점에 해당하는 t+1의 RNN 셀의 입력으로 현재 t에서의 은닉 상태를 입력으로 보냅니다.
- RNN 챕터에서도 언급했지만, 이런 구조에서 현재 시점 t에서의 은닉 상태는 과거 시점의 동일한 RNN 셀에서의 모든 은닉 상태의 값들의 영향을 누적해서 받아온 값이라고 할 수 있습니다. 그렇기 때문에 앞서 우리가 언급했던 컨텍스트 벡터는 사실 인코더에서의 마지막 RNN 셀의 은닉 상태값을 말하는 것이며, 이는 입력 문장의 모든 단어 토큰들의 정보를 요약해서 담고있다고 할 수 있습니다.
- 디코더는 인코더의 마지막 RNN 셀의 은닉 상태인 컨텍스트 벡터를 첫번째 은닉 상태의 값으로 사용합니다. 디코더의 첫번째 RNN 셀은 이 첫번째 은닉 상태의 값과, 현재 t에서의 입력값인 `<sos>`로부터, 다음에 등장할 단어를 예측합니다.

#### Teacher Forcing
- 모델을 설계하기 전에 혹시 의아한 점은 없으신가요? 현재 시점의 디코더 셀의 입력은 오직 이전 디코더 셀의 출력을 입력으로 받는다고 설명하였는데 dec_input이 왜 필요할까요?
- 훈련 과정에서는 이전 시점의 디코더 셀의 출력을 현재 시점의 디코더 셀의 입력으로 넣어주지 않고, 이전 시점의 실제값을 현재 시점의 디코더 셀의 입력값으로 하는 방법을 사용할 겁니다. 그 이유는 이전 시점의 디코더 셀의 예측이 틀렸는데 이를 현재 시점의 디코더 셀의 입력으로 사용하면 현재 시점의 디코더 셀의 예측도 잘못될 가능성이 높고 이는 연쇄 작용으로 디코더 전체의 예측을 어렵게 합니다. 이런 상황이 반복되면 훈련 시간이 느려집니다. 만약 이 상황을 원하지 않는다면 이전 시점의 디코더 셀의 예측값 대신 실제값을 현재 시점의 디코더 셀의 입력으로 사용하는 방법을 사용할 수 있습니다. 이와 같이 RNN의 모든 시점에 대해서 이전 시점의 예측값 대신 실제값을 입력으로 주는 방법을 교사 강요라고 합니다.

In [26]:
model_path = os.path.join(my_path, f"fra_eng_char-level_seq2seq-2.h5")
if os.path.exists(model_path):
    model = tf.keras.models.load_model(model_path)

    model.summary()
else:
    input_enc = Input(shape=(None, len(char2idx_src)+1), name="input_enc")
    _, h_state, c_state = LSTM(units=256, return_state=True, name="lstm_enc")(input_enc)

    input_dec = Input(shape=(None, len(char2idx_tar)+1), name="input_dec")
    lstm_dec_layer = LSTM(units=256, return_sequences=True, return_state=True, name="lstm_dec")
    lstm_dec, _, _ = lstm_dec_layer(input_dec, initial_state=[h_state, c_state])
    # 디코더의 첫 상태를 인코더의 은닉 상태, 셀 상태로 합니다.
    dense_dec_layer = Dense(units=len(char2idx_tar)+1, activation="softmax", name="dense_dec")
    dense_dec = dense_dec_layer(lstm_dec)

    model = Model(inputs=[input_enc, input_dec], outputs=dense_dec)

    model.summary()
    
    model.compile(optimizer="rmsprop", loss="categorical_crossentropy",
                  metrics=["categorical_accuracy"])

    es = EarlyStopping(monitor="val_loss", mode="auto", verbose=1, patience=4)
    mc = ModelCheckpoint(filepath=model_path, monitor="val_categorical_accuracy",
                         mode="auto", verbose=1, save_best_only=True)
    
    hist = model.fit(x=[enc_input, dec_input], y=dec_true, batch_size=128, epochs=50,
                     validation_split=0.2, callbacks=[es, mc])

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input_enc (InputLayer)          [(None, None, 59)]   0                                            
__________________________________________________________________________________________________
Input_dec (InputLayer)          [(None, None, 78)]   0                                            
__________________________________________________________________________________________________
LSTM_enc (LSTM)                 [(None, 256), (None, 323584      Input_enc[0][0]                  
__________________________________________________________________________________________________
LSTM_dec (LSTM)                 [(None, None, 256),  343040      Input_dec[0][0]                  
                                                                 LSTM_enc[0][1]             

In [27]:
# 학습이 맞게 됐는지 확인
i = 110
pred = model.predict([tf.expand_dims(enc_input[i], axis=0), tf.expand_dims(dec_input[i], axis=0)])

sent = ""
for idx in tf.argmax(dec_true[i], axis=1).numpy():
    if idx != 0:
        sent += idx2char_tar[idx]
print(sent)

sent = ""
for idx in tf.argmax(pred[0], axis=1).numpy():
    if idx != 0:
        sent += idx2char_tar[idx]
print(sent)

je suis contre ce projet de loi.<EOS>
œ1ll1zp1lp/j90999 9,  + ,,, 9,3ê9


### (?) seq2seq 기계 번역기 동작시키기
- 앞서 seq2seq는 훈련할 때와 동작할 때의 방식이 다르다고 언급한 바 있습니다. 이번에는 입력한 문장에 대해서 기계 번역을 하도록 모델을 조정하고 동작시켜보도록 하겠습니다.
- 전체적인 번역 동작 단계를 정리하면 아래와 같습니다.
    1. 번역하고자 하는 입력 문장이 인코더에 들어가서 은닉 상태와 셀 상태를 얻습니다.
    2. 상태와 `<SOS>`를 디코더로 보냅니다.
    3. 디코더가 `<EOS>`가 나올 때까지 다음 문자를 예측하는 행동을 반복합니다.

- 우선 인코더를 정의합니다. enc_inputs와 encoder_states는 훈련 과정에서 이미 정의한 것들을 재사용하는 것입니다. 이제 디코더를 설계해보겠습니다.

In [28]:
model.layers

In [29]:
input_enc = model.layers[0].output
_, h_state, c_state = model.layers[2].output

enc_model = tf.keras.Model(inputs=input_enc, outputs=[h_state, c_state])

input_dec = model.layers[1].output
h_state_bef = Input(shape=(256,))
c_state_bef = Input(shape=(256,))
# 문장의 다음 단어를 예측하기 위해서 initial_state를 이전 시점의 상태로 사용합니다.
lstm_dec_layer = model.layers[3]
lstm_dec, h_state_aft, c_state_aft = lstm_dec_layer(input_dec, initial_state=[h_state_bef, c_state_bef])
dense_dec_layer = model.layers[4]
dense_dec = dense_dec_layer(lstm_dec)

dec_model = tf.keras.Model(inputs=[input_dec]+[h_state_bef, c_state_bef], outputs=[dense_dec]+[h_state_aft, c_state_aft])

In [49]:
def decode_seq(input_seq):
# seq = enc_input[i:i+1]
    enc_states = enc_model.predict(input_seq)

    # <SOS>에 해당하는 OHE를 생성합니다.
    seq = np.zeros((1, 1, len(char2idx_tar)+1))
    seq[0, 0, char2idx_tar["<SOS>"]] = 1

    stop_cond = False
    decoded_sent = ""
    # stop_cond이 True가 될 때까지 반복합니다.
    while not stop_cond:
        # 이점 시점의 states를 현재 시점의 states로 사용합니다.
        output_tokens, h_state, c_state = dec_model.predict([seq] + enc_states)
        argmax = np.argmax(output_tokens[0, -1, :])
    #     argmax = np.argmax(output_tokens[0, 0])
        char = idx2char_tar[argmax]
        decoded_sent += char
        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장합니다.
        seq = np.zeros((1, 1, len(char2idx_tar)+1))
        seq[0, 0, argmax] = 1
        enc_states = [h_state, c_state]
        
        # "<EOS>"에 도달하거나 최대 길이를 넘으면 stop_cond=True를 저장합니다.
        if char == "<EOS>" or len(decoded_sent) == max_len_dec:
            stop_cond = True
    return decoded_sent

In [53]:
actual, pred = list(), list()
for seq_index in range(231, 236):
    input_seq = enc_input[seq_index:seq_index+1]
    decoded_sent = decode_seq(input_seq)
    
    actual.append([data["tar"][seq_index][1:len(data["tar"][seq_index])-1].split()])
    pred.append(decoded_sent[:len(decoded_sent)-1].split())
                  
    print(35 * "-")
    print(f"입력 문장 : {data['src'][seq_index]}")
    print(f"정답 문장 : {data['tar'][seq_index][1:len(data['tar'][seq_index])-1]}")
    print(f"번역 문장 : {decoded_sent[:len(decoded_sent)-1]}")
    sf = SmoothingFunction()
    print(f"BLEU-1 : {corpus_bleu(actual, pred, weights=(1, 0, 0, 0),\
                                  smoothing_function=sf.method1)}")
    print(f"BLEU-2 : {corpus_bleu(actual, pred, weights=(1/2, 1/2, 0, 0),\
                                  smoothing_function=sf.method1)}")
    print(f"BLEU-3 : {corpus_bleu(actual, pred, weights=(1/3, 1/3, 1/3, 0),\
                                  smoothing_function=sf.method1)}")
    print(f"BLEU-4 : {corpus_bleu(actual, pred, weights=(1/4, 1/4, 1/4, 1/4),\
                                  smoothing_function=sf.method1)}")

-----------------------------------
입력 문장 : why do you need change?
정답 문장 : ourquoi as-tu besoin de changement 
번역 문장 : jz3lw(++1yj1…w+l1+jl…zp3+lœ1ljz… <EOS
BLEU-1 : 0
BLEU-2 : 0
BLEU-3 : 0
BLEU-4 : 0
-----------------------------------
입력 문장 : why did you change your mind?
정답 문장 : ourquoi as-tu changé d'avis 
번역 문장 : j(l+z0êp+lœ;(s1êjl3;(lw(+l8j8ljê(s(p//8 <EOS
BLEU-1 : 0
BLEU-2 : 0
BLEU-3 : 0
BLEU-4 : 0
-----------------------------------
입력 문장 : we don't want to lose you.
정답 문장 : ous ne voulons pas vous perdre
번역 문장 : sz0+l(s1)ljz0jlœ1l+0pj1lùl/(l…(p+z3 <EOS
BLEU-1 : 0
BLEU-2 : 0
BLEU-3 : 0
BLEU-4 : 0
-----------------------------------
입력 문장 : see that this never happens again.
정답 문장 : aites en sorte que ça ne se produise plus
번역 문장 : èj1+ysz0+lwêèj1lùlc(pê1lh(l&<EOS
BLEU-1 : 0
BLEU-2 : 0
BLEU-3 : 0
BLEU-4 : 0
-----------------------------------
입력 문장 : you must not lose sight of your main object.
정답 문장 : l ne faut pas que tu perdes de vue ton objectif principal
번역 문장 : /1+lwêp4l+